### Import and Load model

In [2]:
import time
import sys
sys.path.append("../")
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np 
from scipy.stats import entropy
from tqdm import tqdm

import torch
from torch import nn
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from bayesian_torch.models.dnn_to_bnn import dnn_to_bnn

from utils.plots import plot_pr,plot_roc, plot_dist, plot_violin
from utils.metircs import accuracy,mutual_info,nll, ece,predictive_entropy
from utils.visual import ProgressMeter, AverageMeter, Summary
from data_utils.get_datasets import get_dataset
from model_utils.get_models import get_model
from uncertainty.mc_dropout import mc_dropout_predict
from uncertainty.test_time_aug import test_time_aug_predict
from uncertainty.deep_ensemble import deep_ensembel_predict,EnsembleModel
from uncertainty.baseline_softmax import baseline_softmax_predict
from uncertainty.bnn_svi import bnn_svi_predict
from adversarial.adversarial_attack import fgsm_attack, pgd_attack

In [3]:
datanames = ["cifar10","mnist","fashionmnist","svhn"]
model = get_model("vgg16", 10)
const_bnn_prior_parameters = {
    "prior_mu": 0.0,
    "prior_sigma": 1.0,
    "posterior_mu_init": 0.0,
    "posterior_rho_init": -3.0,
    "type":  "Reparameterization",## Flipout or Reparameterization
    "moped_enable": "",  # initialize mu/sigma from the dnn weights
    "moped_delta": 0.5,
}
device = torch.device('cuda:0')
dnn_to_bnn(model, const_bnn_prior_parameters)
model = model.to(device)
model.eval()
checkpoint = torch.load("../saved_models/bayesian/vgg16/2023_11_28_21_33_05/vgg16_best_model_92.31.pth",map_location=device)
model.load_state_dict(checkpoint['state_dict'])

use private models...


<All keys matched successfully>

In [4]:
# summary(model,(3,32,32))
# print(model)

In [5]:
#random Gaussian noise fed into an image classifier gives a “prediction confidence” or predicted class probability of 91%
noise = torch.randn(64,3,32,32).to(device)
prob = torch.softmax(model(noise), axis=1)
print(torch.max(prob,axis=1))

torch.return_types.max(
values=tensor([0.4156, 0.5915, 0.7408, 0.9245, 0.4962, 0.8783, 0.5142, 0.6998, 0.7760,
        0.7312, 0.7220, 0.6941, 0.7307, 0.4772, 0.5773, 0.6844, 0.7407, 0.7431,
        0.4427, 0.5733, 0.5509, 0.8667, 0.7367, 0.4730, 0.5352, 0.5955, 0.7185,
        0.8187, 0.6764, 0.5592, 0.7979, 0.5654, 0.8707, 0.4014, 0.4847, 0.7766,
        0.7087, 0.6925, 0.7018, 0.6285, 0.4790, 0.8056, 0.7672, 0.8861, 0.5037,
        0.4441, 0.6716, 0.3626, 0.9086, 0.4482, 0.5696, 0.6920, 0.5640, 0.5631,
        0.5509, 0.5653, 0.6982, 0.7559, 0.6781, 0.7524, 0.5209, 0.3610, 0.6003,
        0.8032], device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 2, 6, 6, 6, 6, 6, 6, 6, 6, 2,
        6, 6, 6, 6, 6, 6, 6, 4, 6, 4, 2, 6, 6, 6, 6, 2, 6, 6, 6, 6, 2, 2, 6, 2,
        6, 2, 6, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 6, 6], device='cuda:0'))


In [6]:
%reload_ext autoreload
%autoreload 2

### BNN SVI

In [ ]:
datanames = ["cifar10","mnist", "fashionmnist","svhn"]
probs_list = []
targets_list = []
for dataname in datanames:
    val_transform = transforms.Compose(
    [
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(
                (0.4914, 0.4822, 0.4465),
                (0.2023, 0.1994, 0.2010))
    ]
    )
    if dataname=="mnist" or dataname=="fashionmnist":
        val_transform.transforms.insert(2,transforms.Lambda(lambda x:x.repeat(3,1,1)))
    _, val_dataset = get_dataset(dataname,"../data",None,val_transform)
    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=128, shuffle=False,
        num_workers=4, pin_memory=True)

    probs, targets = bnn_svi_predict(val_loader, model, device, 20)
    probs_list.append(probs.cpu().detach().numpy())
    targets_list.append(targets.cpu().detach().numpy())

 * Time 0.353 Acc@1 91.940


#### misclassified

In [ ]:
print(probs_list[0].shape)

In [ ]:
cifar10_mi =  mutual_info(probs_list[0])*-1
cifar10_probs = np.mean(probs_list[0],axis=1)
cifar10_target = targets_list[0]

In [ ]:
cifar10_pred = np.argmax(cifar10_probs,axis=1)
labels = (cifar10_pred == cifar10_target).astype(int)#正确分类标为1，误分类标为0
random_pred = np.random.rand(labels.shape[0])

In [ ]:
succ_mi_mean = cifar10_mi[labels==1].mean()
err_mi_mean = cifar10_mi[labels==0].mean()
print(f"正确分类样本数{labels.sum()},平均MI:{succ_mi_mean:.3f}\n错误分类样本数{(1-labels).sum()},平均MI:{err_mi_mean:.3f}")

In [ ]:
cifar10_ent = predictive_entropy(probs_list[0])*-1
succ_ent_mean = cifar10_ent[labels==1].mean()
err_ent_mean = cifar10_ent[labels==0].mean()
print(f"正确分类样本数{labels.sum()},平均AU:{succ_ent_mean:.3f}\n错误分类样本数{(1-labels).sum()},平均AU:{err_ent_mean:.3f}")

In [ ]:
fig, axes = plt.subplots(2,3, figsize=(20, 12))
plot_roc(axes[0][0], labels,cifar10_mi,"mi/succ")
plot_pr(axes[0][1],labels,random_pred,"random/succ")
plot_pr(axes[0][2],labels,cifar10_mi,"mi/succ")
plot_roc(axes[1][0], labels,cifar10_ent,"entropy/succ")
plot_pr(axes[1][1],labels,random_pred,"random/succ")
plot_pr(axes[1][2],labels,cifar10_ent,"entropy/succ")
plt.tight_layout()

In [ ]:
labels2 = (cifar10_pred != cifar10_target).astype(int)#误分类标为1，正确分类标为0
cifar10_mi2 = -cifar10_mi
random_pred2 = -random_pred
cifar10_ent2 = -cifar10_ent 

In [ ]:
fig, axes = plt.subplots(2,3, figsize=(20, 12))
plot_roc(axes[0][0], labels2,cifar10_mi2,"mi/err")
plot_pr(axes[0][1],labels2,random_pred2,"random/err")
plot_pr(axes[0][2],labels2,cifar10_mi2,"mi/err")
plot_roc(axes[1][0], labels2,cifar10_ent2,"entropy/err")
plot_pr(axes[1][1],labels2,random_pred2,"random/err")
plot_pr(axes[1][2],labels2,cifar10_ent2,"entropy/err")
plt.tight_layout()

#### OOD

In [ ]:
mi_list = []
for i, probs in enumerate(probs_list):
    mi = mutual_info(probs)
    mi_list.append(mi)

In [ ]:
ent_list = []
for i, probs in enumerate(probs_list):
    ent = predictive_entropy(probs)
    ent_list.append(ent)

In [ ]:
#绘制mi,熵的分布
fig, axes = plt.subplots(1,2, figsize=(13, 6))
plot_dist(axes[0],mi_list,datanames,"mi")
plot_dist(axes[1], ent_list,datanames,"entropy")
plt.tight_layout()

In [ ]:
#绘制小提琴图
fig, axes = plt.subplots(1,4, figsize=(25, 6))
plot_violin(axes,mi_list,datanames,"mi")
fig, axes = plt.subplots(1,4, figsize=(25, 6))
plot_violin(axes, ent_list,datanames,"entropy")
plt.tight_layout()

In [ ]:
#IND样本视为正样本， mnist样本视为负样本
ood_idx=1
print(f"cifar10 test nums:{len(mi_list[0])},mnist test nums:{len(mi_list[1])}")
labels = [1]*len(mi_list[0])+[0]*len(mi_list[ood_idx])
mi = np.concatenate([mi_list[0],mi_list[ood_idx]])*-1
ent = np.concatenate([ent_list[0],ent_list[ood_idx]])*-1
random_pred = np.random.rand(len(labels))*-1

In [ ]:
fig, axes = plt.subplots(2,3, figsize=(20, 12))
plot_roc(axes[0][0], labels,mi,"mi/ind")
plot_pr(axes[0][1],labels,random_pred,"random/ind")
plot_pr(axes[0][2],labels,mi,"mi/ind")
plot_roc(axes[1][0], labels,ent,"entropy/ind")
plot_pr(axes[1][1],labels,random_pred,"random/ind")
plot_pr(axes[1][2],labels,ent,"entropy/ind")
plt.tight_layout()

In [ ]:
#IND样本视为负样本， mnist样本视为正样本
labels2 = [0]*len(mi_list[0])+[1]*len(mi_list[1])
mi2 =  -mi
ent2 = -ent
random_pred2 = -random_pred

In [ ]:
fig, axes = plt.subplots(2,3, figsize=(20, 12))
plot_roc(axes[0][0], labels2,mi2,"mi/ood")
plot_pr(axes[0][1],labels2,random_pred2,"random/ood")
plot_pr(axes[0][2],labels2,mi2,"mi/ood")
plot_roc(axes[1][0], labels2,ent2,"entropy/ood")
plot_pr(axes[1][1],labels2,random_pred2,"random/ood")
plot_pr(axes[1][2],labels2,ent2,"entropy/ood")
plt.tight_layout()

#### advasarial attack

In [ ]:
_, val_dataset = get_dataset("cifar10","../data",None,val_transform)
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=1, shuffle=False,
    num_workers=4, pin_memory=True)

In [ ]:
adv_probs_list = []
origin_probs_list = []
method = "pgd"
epsilon = 0.01
correct = 0
err = 0
num_correct=0
num_monte_carlo = 20
for images, targets in tqdm(val_loader):
    images, targets = images.to(device), targets.to(device)
    origin_outputs = []
    for _ in range(num_monte_carlo):# add bnn_svi
        prob  = torch.softmax(model.forward(images),dim=1)#输出的概率
        origin_outputs.append(prob)
    origin_outputs = torch.stack(origin_outputs, dim=0)
    origin_prob_mean = torch.mean(origin_outputs, dim=0)
    _, origin_pred = torch.max(origin_prob_mean,axis=1)
    # num_correct += (origin_pred == targets).sum().item()
    if origin_pred.item() != targets.item():
        err += 1
        continue

    if method == "fgsm":
        perturbed_images = fgsm_attack(model, images, targets, epsilon)
    elif method == "pgd":
        perturbed_images = pgd_attack(model, images, targets, epsilon)

    adv_outputs = []
    for _ in range(num_monte_carlo):# add bnn_svi
        prob  = torch.softmax(model.forward(images),dim=1)#输出的概率
        adv_outputs.append(prob)
    adv_outputs = torch.stack(adv_outputs, dim=0)
    adv_prob_mean = torch.mean(adv_outputs, dim=0)
    _, adv_pred = adv_prob_mean.max(1, keepdim=True)
    adv_probs_list.append(adv_outputs.cpu().detach().numpy())
    origin_probs_list.append(origin_outputs.cpu().detach().numpy())

    if adv_pred.item() == targets.item():
        correct += 1

adv_acc = correct/float(len(val_loader)-err)
print(
    f"Epsilon: {epsilon}\tTest Accuracy = {correct} / {len(val_loader)} = {adv_acc}\tIgnore err = {err}")

In [ ]:
adv_probs = np.concatenate(adv_probs_list,axis=1).transpose(1,0,2)
origin_probs = np.concatenate(origin_probs_list,axis=1).transpose(1,0,2)
print(adv_probs.shape,origin_probs.shape)

In [ ]:
mi_list = []
for i, probs in enumerate([origin_probs,adv_probs]):
    mi_list.append(mutual_info(probs))

In [ ]:
ent_list = []
for i, probs in enumerate([origin_probs,adv_probs]):
    ent = predictive_entropy(probs)
    ent_list.append(ent)

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(13, 6))
plot_dist(axes[0],mi_list,["original","advasarial"],"mi")
plot_dist(axes[1], ent_list,["original","advasarial"],"entropy")
plt.tight_layout()

In [ ]:
#原始样本视为正样本， 攻击样本视为负样本
labels = [1]*len(mi_list[0])+[0]*len(mi_list[1])
mi = np.concatenate([mi_list[0],mi_list[1]])*-1
ent = np.concatenate([ent_list[0],ent_list[1]])*-1
random_pred = np.random.rand(len(labels))*-1

In [ ]:
fig, axes = plt.subplots(2,3, figsize=(20, 12))
plot_roc(axes[0][0], labels,mi,"mi/origin")
plot_pr(axes[0][1],labels,random_pred,"random/origin")
plot_pr(axes[0][2],labels,mi,"mi/origin")
plot_roc(axes[1][0], labels,ent,"entropy/origin")
plot_pr(axes[1][1],labels,random_pred,"random/origin")
plot_pr(axes[1][2],labels,ent,"entropy/origin")
plt.tight_layout()

In [ ]:
#原始样本视为正样本， 攻击样本视为负样本
labels2 = [0]*len(mi_list[0])+[1]*len(mi_list[1])
mi2 =  -mi
ent2 = -ent
random_pred2 = -random_pred

In [ ]:
fig, axes = plt.subplots(2,3, figsize=(20, 12))
plot_roc(axes[0][0], labels2,mi2,"mi/adv")
plot_pr(axes[0][1],labels2,random_pred2,"random/adv")
plot_pr(axes[0][2],labels2,mi2,"mi/adv")
plot_roc(axes[1][0], labels2,ent2,"entropy/adv")
plot_pr(axes[1][1],labels2,random_pred2,"random/adv")
plot_pr(axes[1][2],labels2,ent2,"entropy/adv")
plt.tight_layout()